# Riddler Classic 2022-04-08

https://fivethirtyeight.com/features/can-you-be-mediocre-enough-to-win/

*This week’s Classic also comes from Ben Orlin, although it’s relatively mediocre compared to the Express.*

*In the three-player Game of Mediocrity, you win by not winning too much.*

*Each round, every player secretly picks a number from 0 to 10. The numbers are simultaneously revealed, and the median number wins that number of points. (If two or more players pick the same number, then the winner is randomly selected from among them.)*

*Player A says 6, B says 3, and C says 2. B is the winner, having said the median, and earns 3 points.
After five rounds, the winner is whoever has the median number of points. (Again, if two or more players have the same score, then the winner is randomly selected from among them.)*

*With one round remaining, players A, B and C have 6, 8 and 10 points, respectively. Player A sighs and writes down “3,” but fails to do so in secret. Players B and C both see player A’s number (and both see that the other saw A’s number), and will take care to write their own numbers in secret. Assuming everyone plays to win, what numbers should B and C choose?*

Obviously, player A has determined that winning is impossible. Thus, player A only serves to hurt player B's chances of winning.

Let's start by calculating conditional probabilities of winning for all of each of the two relevant player's choices.

In [1]:
import statistics
import pandas as pd

In [2]:
#Conditional probabilities for player B's options
for b in range(0, 11):
    success = 0
    for c in range(0, 11):
        median = statistics.median([3, b, c])
        if b == median and b >= 2: 
            success += (0.5 if b == 2 else 1)/[3, b, c].count(median)
    print("P(B wins | B picks " + str(b) + ") =",  str(round(success/11, 2)))

P(B wins | B picks 0) = 0.0
P(B wins | B picks 1) = 0.0
P(B wins | B picks 2) = 0.11
P(B wins | B picks 3) = 0.48
P(B wins | B picks 4) = 0.59
P(B wins | B picks 5) = 0.5
P(B wins | B picks 6) = 0.41
P(B wins | B picks 7) = 0.32
P(B wins | B picks 8) = 0.23
P(B wins | B picks 9) = 0.14
P(B wins | B picks 10) = 0.05


In [3]:
#Conditional probabilities for player C's options
for c in range(0, 11):
    success = 0
    for b in range(0, 11):
        median = statistics.median([3, b, c])
        if b == median and b >= 2: 
            success += (0.5 if b == 2 else 1)/[3, b, c].count(median)
    print("P(C wins | C picks " + str(c) + ") =",  str(round(1 - (success/11), 2)))


P(C wins | C picks 0) = 0.91
P(C wins | C picks 1) = 0.91
P(C wins | C picks 2) = 0.93
P(C wins | C picks 3) = 0.97
P(C wins | C picks 4) = 0.91
P(C wins | C picks 5) = 0.82
P(C wins | C picks 6) = 0.73
P(C wins | C picks 7) = 0.64
P(C wins | C picks 8) = 0.55
P(C wins | C picks 9) = 0.45
P(C wins | C picks 10) = 0.36


If players B and C knew that the other player hadn't seen player A's final number, they would pick 4 and 3, respectively, to optimize their chances of winning. Instead, this becomes a game theory problem, and we need to find the strict Nash equilibrium. With a lead of 2, Player C has the upper hand in this scenario, so the strict Nash equilibrium will be the outcome which gives player C the highest chance of winning assuming player B acts rationally. 

What exactly would rational behavior look like for player B? Well, for starters, player B needs to pick a number ≥ 2 to even have a chance of winning. For each number played by player C, player B will play the number which yields the highest chance of winning. Let's calculate those numbers and probabilities.

In [4]:
df = pd.DataFrame()
for c in range(0, 11):
    E_max = (0, 0)
    for b in range(2, 11):
        median = statistics.median([3, b, c])
        if b == median:
            E = 1 / [3, b, c].count(median) / (2 if b == 2 else 1) 
            if E >= E_max[1]:
                E_max = (b, E)
    df = df.append(pd.DataFrame({"Player C's Pick": [c], "Player B's Optimal Pick": [E_max[0]], "P(B Wins | B Picks Optimally)": [round(E_max[1], 2)]}))

print(df.to_string(index = False))

 Player C's Pick  Player B's Optimal Pick  P(B Wins | B Picks Optimally)
               0                        3                           0.50
               1                        3                           0.50
               2                        3                           0.50
               3                        3                           0.33
               4                        4                           0.50
               5                        4                           1.00
               6                        5                           1.00
               7                        6                           1.00
               8                        7                           1.00
               9                        8                           1.00
              10                        9                           1.00


Again, the strict Nash equilibrium in this scenario is the outcome which gives player C the highest chance of winning assuming player B acts rationally. As shown above, this equilibrium is achieved by all players picking 3. When player C picks 3, the median is set in stone at 3, and the only way player B can win the round (and the game) is by also picking 3 and hoping the random draw goes well. Player B will have a $\frac{1}{3}$ chance of winning, and player C will have a $\frac{2}{3}$ chance. 